<a href="https://colab.research.google.com/github/DanhVanPham/-sample/blob/master/image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install tflearn


In [ ]:

import cv2
import os
import numpy as np
from random import shuffle
from tqdm import tqdm
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense


TRAIN_DIR = '/content/drive/MyDrive/Colab Notebooks/dog_cat_classifier/train'
TEST_DIR = '/content/drive/MyDrive/Colab Notebooks/dog_cat_classifier/test'
IMG_SIZE = 50
LR = 1e-3

MODEL_NAME = 'dogsvscats-{}-{}.model'.format(LR, '6conv-basic')

'''label'''
def label_img(img):
	word_label = img.split('.')[-3]

	if word_label == 'cat': return [1, 0]
	elif word_label == 'dog': return [0, 1]

'''Creating the training data'''
def create_train_data():
	training_data = []

	for img in tqdm(os.listdir(TRAIN_DIR)):
		# labeling the images
		label = label_img(img)
		path = os.path.join(TRAIN_DIR, img)
		img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
		img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
		training_data.append([np.array(img), np.array(label)])

	shuffle(training_data)

	np.save('training_data.npy', training_data)
	return training_data

def process_test_data():
	testing_data = []
	for img in tqdm(os.listdir(TEST_DIR)):
		path = os.path.join(TEST_DIR, img)
		img_num = img.split('.')[0]
		img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
		img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
		testing_data.append([np.array(img), img_num])
		
	shuffle(testing_data)
	np.save('testing_data.npy', testing_data)
	return testing_data

train_data = create_train_data()
test_data = process_test_data()
# train  and test
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

from tensorflow.python.framework import ops
ops.reset_default_graph()
convnet = input_data(shape =[None, IMG_SIZE, IMG_SIZE, 1], name ='input')

convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation ='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation ='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation ='softmax')
convnet = regression(convnet, optimizer ='adam', learning_rate = LR,
	loss ='categorical_crossentropy', name ='targets')

model = tflearn.DNN(convnet, tensorboard_dir ='log')

# Split the testing data and training data
train = train_data[:-500]
test = train_data[-500:]


X = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Y = [i[1] for i in train]
test_x = np.array([i[0] for i in test]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test]


# epoch = 50 taken
model.fit({'input': X}, {'targets': Y}, n_epoch = 50,
	validation_set =({'input': test_x}, {'targets': test_y}),
	snapshot_step = 500, show_metric = True, run_id = MODEL_NAME)
model.save(MODEL_NAME)
# save model
model.save("model.tfl")


# serialize model to JSON
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")

'''Testing the data'''
import matplotlib.pyplot as plt

test_data = np.load('testing_data.npy',allow_pickle=True)

fig = plt.figure()

for num, data in enumerate(test_data[:20]):
	# cat: [1, 0]
	# dog: [0, 1]
	
	img_num = data[1]
	img_data = data[0]
	
	y = fig.add_subplot(4, 5, num + 1)
	orig = img_data
	data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)

	model_out = model.predict([data])[0]
	
	if np.argmax(model_out) == 1: str_label ='Dog'
	else: str_label ='Cat'
		
	y.imshow(orig, cmap ='gray')
	plt.title(str_label)
	y.axes.get_xaxis().set_visible(False)
	y.axes.get_yaxis().set_visible(False)
plt.show()


In [ ]:
model.load("model.tfl")
'''Testing the data'''
'''Testing the data'''
import matplotlib.pyplot as plt
IMG_SIZE_1=50

test_data = np.load('testing_data.npy',allow_pickle=True)

fig = plt.figure()

for num, data in enumerate(test_data[:20]):
	# cat: [1, 0]
	# dog: [0, 1]
	
	img_num = data[1]
	img_data = data[0]
	
	y = fig.add_subplot(4, 5, num + 1)
	orig = img_data
	data = img_data.reshape(IMG_SIZE_1, IMG_SIZE_1, 1)

	model_out = model.predict([data])[0]
	
	if np.argmax(model_out) == 1: str_label ='Dog'
	else: str_label ='Cat'
		
	y.imshow(orig, cmap ='gray')
	plt.title(str_label)
	y.axes.get_xaxis().set_visible(False)
	y.axes.get_yaxis().set_visible(False)
plt.show()